In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab
import scipy.stats as stats
from tqdm.notebook import tqdm
from scipy.stats import ttest_ind


ayu_colors = {
    "axes.facecolor": "#1F2430",  # фон графика
    "figure.facecolor": "#1F2430",  # фон фигуры
    "text.color": "#CBCCC6",  # цвет текста
    "axes.labelcolor": "#CBCCC6",  # цвет подписей осей
    "xtick.color": "#707A8C",  # цвет делений оси X
    "ytick.color": "#707A8C",  # цвет делений оси Y
    "grid.color": "#232834",  # цвет сетки
    "axes.prop_cycle": plt.cycler(
        color=["#8dd3c7", "#feffb3", "#bfbbd9", "#fa8174", "#81b1d2"]
    ),  # цвета линий как в dark_background
}

plt.rcParams.update(ayu_colors)

In [15]:
cards = np.random.binomial(1, 0.5, size=10)
guess = np.random.binomial(1, 0.5, size=10)

In [21]:
int((guess == cards).sum())

2

In [31]:
result = []

for _ in range(1000):
    cards = np.random.binomial(1, 0.5, size=10)
    guess = np.random.binomial(1, 0.5, size=10)
    if all(cards == guess):
        result.append(10)
    elif int((guess == cards).sum()) == 9:
        result.append(9)
    else:
        continue

In [32]:
len(result) / 1000

0.009